In [ ]:
!pip install --upgrade diffusers[torch]
!pip install transformers
!pip install matplotlib
!pip install opencv-python
!pip install -U peft
#!wget "https://raw.githubusercontent.com/huggingface/diffusers/main/examples/community/lpw_stable_diffusion_xl.py"

In [ ]:
from diffusers import LEditsPPPipelineStableDiffusionXL, DDIMScheduler
from diffusers.utils import load_image
import torch
from diffusers import AutoPipelineForText2Image
from accelerate import Accelerator

# Initialize the accelerator
accelerator = Accelerator()

model_id = "stabilityai/stable-diffusion-xl-base-1.0"
# model_id = "GraydientPlatformAPI/ultrium7"
local_lora_model_paths = ["./xpenis263.XgFC.safetensors", "./pein.lMnk.safetensors", "./bbc20XL.fQW3.safetensors", "./hyperpenisfinal.BmyP.safetensors"] 
lora_weights = [1.25, 1.25, 1, 0.5]
local_lora_model_paths = ["./xpenis263.XgFC.safetensors", "./hyperpenisfinal.BmyP.safetensors"]
lora_weights = [1.25,0.15]

scheduler = DDIMScheduler.from_pretrained(model_id, subfolder="scheduler",cache_dir="models")
# pipe = LEditsPPPipelineStableDiffusionXL.from_pretrained(
#     model_id,
#     scheduler=scheduler,
#     cache_dir="models"
# )

pipe = LEditsPPPipelineStableDiffusionXL.from_single_file("./juggernautX.lIDA.safetensors", scheduler=scheduler,cache_dir="models")
pipe.to("cuda")

for lora_model_path, weight in zip(local_lora_model_paths, lora_weights):
    pipe.load_lora_weights(lora_model_path, weight=weight) 

pipe = accelerator.prepare(pipe)


In [2]:
def resize_image_to_max_axis(image, max_dimension=1024):

    # Calculate the new size maintaining the aspect ratio
    width, height = image.size
    if width > height:
        new_width = max_dimension
        new_height = int((height / width) * max_dimension)
    else:
        new_height = max_dimension
        new_width = int((width / height) * max_dimension)

    print((new_width, new_height))
    # Resize the image
    resized_image = image.resize((new_width, new_height), Image.LANCZOS)
    return resized_image


In [ ]:
#Working without embeds, text only - limited by length
import matplotlib.pyplot as plt
from PIL import Image
image = load_image("d.jpg")#.resize((512, 512))
image = resize_image_to_max_axis(image)
image.show()

pipe.invert(
    image = image,
    num_inversion_steps=50,
    skip=0.2
)

positive_prompt = """
long prompt"""

negative_prompt = """
long negative prompt
"""

for i in range(10):
    w = 0.5 + i* 0.25
    print(i, w)
    edited_image = pipe(
        editing_prompt=[positive_prompt, negative_prompt],
        reverse_editing_direction=[False, True],
        edit_guidance_scale=[w,w],
        edit_threshold=[0.9,0.9],
    ).images[0]
    
    edited_image.save(f"result{i}.png")
    edited_image.show()
    # Display the edited image
    

<span style="font-size: 36px; font-weight: bold;">FAILED</span>
<br>
<span style="font-size: 16px;">Failed attempt and work in progress</span>


**Unsolved error: UnboundLocalError: local variable 'edit_concepts_embeds' referenced before assignment**

**For longer texts there is a trick to pass embeds, this fails with the pipeline**


In [ ]:

import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from lpw_stable_diffusion_xl import get_weighted_text_embeddings_sdxl
import os
from datetime import datetime
from PIL import Image, PngImagePlugin
import gc
import random
import torch
import time 

image = load_image("d.jpg")#.resize((512, 512))
image = resize_image_to_max_axis(image)
image.show()

prompt = """
long
"""

negative_prompt = """
short
"""

prompt_embeds,negative_prompt_embeds,pooled_prompt_embeds,negative_pooled_prompt_embeds = get_weighted_text_embeddings_sdxl(pipe, prompt = prompt, neg_prompt = negative_prompt)

# prompt_embeds[0] -= negative_prompt_embeds[0]
prompt_embeds -= negative_prompt_embeds
pooled_prompt_embeds -= negative_pooled_prompt_embeds

pipe.invert(
    image = image,
    num_inversion_steps=50,
    skip=0.2
)
print(len(prompt_embeds))
for i in range(20):
    w = 7.5 - i 
    print(i, w)
    edited_image = pipe(editing_prompt=[prompt, negative_prompt],
        reverse_editing_direction=[False, True],
        editing_prompt_embeddings = prompt_embeds,
        editing_pooled_prompt_embeddings = pooled_prompt_embeds,
        edit_guidance_scale=w,
        edit_threshold=0.9,
        enable_edit_guidance=True
    ).images[0]
    
    edited_image.save(f"result{i}.png")
    edited_image.show()
    # Display the edited image
    

# Failed to fill black or random 

In [ ]:

import numpy as np
from PIL import Image

def fill_black_with_random_color(image_path, output_path):
    # Load the image with PIL
    pil_image = Image.open(image_path)
    
    # Convert the image to a NumPy array for OpenCV processing
    image_np = np.array(pil_image)
    
    # Identify black areas (thresholding)
    black_areas = np.all(image_np == [0, 0, 0], axis=-1)
    
    # Create a mask of the same shape
    random_colors = np.random.randint(0, 256, size=(image_np.shape[0], image_np.shape[1], 3), dtype=np.uint8)
    
    # Fill the black areas with random colors
    image_np[black_areas] = random_colors[black_areas]
    
    # Convert the NumPy array back to a PIL image
    result_image = Image.fromarray(image_np)
    
    # Save the result
    result_image.save(output_path)

# Usage
input_image_path = 'image_0.png'
output_image_path = 'output_image.jpg'
fill_black_with_random_color(input_image_path, output_image_path)
